# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f00bc069c70>
├── 'a' --> tensor([[-0.3325, -0.1795,  1.3716],
│                   [ 1.8039, -2.3693, -2.4119]])
└── 'x' --> <FastTreeValue 0x7f00bc0694c0>
    └── 'c' --> tensor([[-1.0785, -1.9763, -0.6128, -0.0162],
                        [-0.7942, -0.3993,  0.5239, -0.4707],
                        [-0.4826, -1.9290,  0.2433, -0.8819]])

In [4]:
t.a

tensor([[-0.3325, -0.1795,  1.3716],
        [ 1.8039, -2.3693, -2.4119]])

In [5]:
%timeit t.a

65.2 ns ± 1.24 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f00bc069c70>
├── 'a' --> tensor([[ 1.5440, -0.6215, -0.3965],
│                   [-1.8599, -1.0966,  0.7096]])
└── 'x' --> <FastTreeValue 0x7f00bc0694c0>
    └── 'c' --> tensor([[-1.0785, -1.9763, -0.6128, -0.0162],
                        [-0.7942, -0.3993,  0.5239, -0.4707],
                        [-0.4826, -1.9290,  0.2433, -0.8819]])

In [7]:
%timeit t.a = new_value

81.4 ns ± 3.52 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3325, -0.1795,  1.3716],
               [ 1.8039, -2.3693, -2.4119]]),
    x: Batch(
           c: tensor([[-1.0785, -1.9763, -0.6128, -0.0162],
                      [-0.7942, -0.3993,  0.5239, -0.4707],
                      [-0.4826, -1.9290,  0.2433, -0.8819]]),
       ),
)

In [10]:
b.a

tensor([[-0.3325, -0.1795,  1.3716],
        [ 1.8039, -2.3693, -2.4119]])

In [11]:
%timeit b.a

69 ns ± 1.58 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1921, -1.1560,  0.7869],
               [ 0.4271,  0.7530,  0.2094]]),
    x: Batch(
           c: tensor([[-1.0785, -1.9763, -0.6128, -0.0162],
                      [-0.7942, -0.3993,  0.5239, -0.4707],
                      [-0.4826, -1.9290,  0.2433, -0.8819]]),
       ),
)

In [13]:
%timeit b.a = new_value

603 ns ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.07 µs ± 18.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.8 µs ± 284 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

253 µs ± 8.97 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

249 µs ± 8.28 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f00062fe730>
├── 'a' --> tensor([[[-0.3325, -0.1795,  1.3716],
│                    [ 1.8039, -2.3693, -2.4119]],
│           
│                   [[-0.3325, -0.1795,  1.3716],
│                    [ 1.8039, -2.3693, -2.4119]],
│           
│                   [[-0.3325, -0.1795,  1.3716],
│                    [ 1.8039, -2.3693, -2.4119]],
│           
│                   [[-0.3325, -0.1795,  1.3716],
│                    [ 1.8039, -2.3693, -2.4119]],
│           
│                   [[-0.3325, -0.1795,  1.3716],
│                    [ 1.8039, -2.3693, -2.4119]],
│           
│                   [[-0.3325, -0.1795,  1.3716],
│                    [ 1.8039, -2.3693, -2.4119]],
│           
│                   [[-0.3325, -0.1795,  1.3716],
│                    [ 1.8039, -2.3693, -2.4119]],
│           
│                   [[-0.3325, -0.1795,  1.3716],
│                    [ 1.8039, -2.3693, -2.4119]]])
└── 'x' --> <FastTreeValue 0x7f00c0148d60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

44 µs ± 1.11 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0017c2ca30>
├── 'a' --> tensor([[-0.3325, -0.1795,  1.3716],
│                   [ 1.8039, -2.3693, -2.4119],
│                   [-0.3325, -0.1795,  1.3716],
│                   [ 1.8039, -2.3693, -2.4119],
│                   [-0.3325, -0.1795,  1.3716],
│                   [ 1.8039, -2.3693, -2.4119],
│                   [-0.3325, -0.1795,  1.3716],
│                   [ 1.8039, -2.3693, -2.4119],
│                   [-0.3325, -0.1795,  1.3716],
│                   [ 1.8039, -2.3693, -2.4119],
│                   [-0.3325, -0.1795,  1.3716],
│                   [ 1.8039, -2.3693, -2.4119],
│                   [-0.3325, -0.1795,  1.3716],
│                   [ 1.8039, -2.3693, -2.4119],
│                   [-0.3325, -0.1795,  1.3716],
│                   [ 1.8039, -2.3693, -2.4119]])
└── 'x' --> <FastTreeValue 0x7f00062ecfd0>
    └── 'c' --> tensor([[-1.0785, -1.9763, -0.6128, -0.0162],
                        [-0.7942, -0.3993,  0.5239, -0.4707],
                 

In [23]:
%timeit t_cat(trees)

41.2 µs ± 802 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

99.1 µs ± 1.61 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.0785, -1.9763, -0.6128, -0.0162],
                       [-0.7942, -0.3993,  0.5239, -0.4707],
                       [-0.4826, -1.9290,  0.2433, -0.8819]],
              
                      [[-1.0785, -1.9763, -0.6128, -0.0162],
                       [-0.7942, -0.3993,  0.5239, -0.4707],
                       [-0.4826, -1.9290,  0.2433, -0.8819]],
              
                      [[-1.0785, -1.9763, -0.6128, -0.0162],
                       [-0.7942, -0.3993,  0.5239, -0.4707],
                       [-0.4826, -1.9290,  0.2433, -0.8819]],
              
                      [[-1.0785, -1.9763, -0.6128, -0.0162],
                       [-0.7942, -0.3993,  0.5239, -0.4707],
                       [-0.4826, -1.9290,  0.2433, -0.8819]],
              
                      [[-1.0785, -1.9763, -0.6128, -0.0162],
                       [-0.7942, -0.3993,  0.5239, -0.4707],
                       [-0.4826, -1.9290,  0.2433, -0.8819]],

In [26]:
%timeit Batch.stack(batches)

96.4 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.0785, -1.9763, -0.6128, -0.0162],
                      [-0.7942, -0.3993,  0.5239, -0.4707],
                      [-0.4826, -1.9290,  0.2433, -0.8819],
                      [-1.0785, -1.9763, -0.6128, -0.0162],
                      [-0.7942, -0.3993,  0.5239, -0.4707],
                      [-0.4826, -1.9290,  0.2433, -0.8819],
                      [-1.0785, -1.9763, -0.6128, -0.0162],
                      [-0.7942, -0.3993,  0.5239, -0.4707],
                      [-0.4826, -1.9290,  0.2433, -0.8819],
                      [-1.0785, -1.9763, -0.6128, -0.0162],
                      [-0.7942, -0.3993,  0.5239, -0.4707],
                      [-0.4826, -1.9290,  0.2433, -0.8819],
                      [-1.0785, -1.9763, -0.6128, -0.0162],
                      [-0.7942, -0.3993,  0.5239, -0.4707],
                      [-0.4826, -1.9290,  0.2433, -0.8819],
                      [-1.0785, -1.9763, -0.6128, -0.0162],
                   

In [28]:
%timeit Batch.cat(batches)

173 µs ± 3.84 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

536 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
